In [5]:
#Loading required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time

In [95]:
#getting list of Districts
url = "https://www.hscap.kerala.gov.in/school.php"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
districts = [item.text for item in soup.find('select', {'name':'cmbdist'}).find_all('option') if item.text not in ['[SELECT]', 'OTHERS']]
districts

['THIRUVANANTHAPURAM',
 'KOLLAM',
 'PATHANAMTHITTA',
 'ALAPPUZHA',
 'KOTTAYAM',
 'IDUKKY',
 'ERNAKULAM',
 'THRISSUR',
 'PALAKKAD',
 'KOZHIKKODE',
 'MALAPPURAM',
 'WAYANAD',
 'KANNUR',
 'KASARGODE']

In [7]:
driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')

C:\Users\pk\AppData\Local\Temp/ipykernel_7092/1144362777.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/chromedriver.exe')


In [143]:
driver.get(url)

In [157]:
#creating master dataframe
master = pd.DataFrame()

In [162]:
#iterating over each districts
for district in districts:
    Select(driver.find_element('name', 'cmbdist')).select_by_visible_text(district)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table_html = soup.find('table', {'id':'schoolListTab'})
    table_pd = pd.read_html(str(table_html))[0]
    if master.shape[0] != 0:
        master = master.append(table_pd)
    else:
        master = table_pd

In [163]:
master

,School Code,"School Name, Address, Phone No",School Type,Local Body Taluk Name,Educational District,Assembly Constituency,Course Code,School Gender,Second Language
0,1001,GOVT MODEL BOYS HSS ATTINGAL Phone No: 0470262...,Government,Attingal (M)Chirayinkeezhu,Attingal,Attingal,11039,Co-Education,12
1,1002,"GOVT. GIRLS HSS, COTTONHILL ,TRIVANDRUM Phone ...",Government,Thiruvananthapuram (C)Thiruvananthapuram,Thiruvananthapuram,Thiruvananthapuram,125101339,Girls,12411
2,1003,"GOVT. V.H.S.S FOR THE DEAF, JAGATHY, TRIVANDRU...",Special,Thiruvananthapuram (C)Thiruvananthapuram,Thiruvananthapuram,Thiruvananthapuram,11,Co-Education,1
3,1004,"DR AMMR HSS ,KATTELA, SREEKARIAM P.O, TRIVANDR...",Residential,Thiruvananthapuram (C)Thiruvananthapuram,Thiruvananthapuram,Thiruvananthapuram,139,Girls,21
4,1005,"GOVT HSS, KILIMANOOR, TRIVANDRUM Phone No: 047...",Government,KilimanoorChirayinkeezhu,Attingal,Attingal,11036,Co-Education,12
...,...,...,...,...,...,...,...,...,...
101,14113,"SATHSS, Manjeswar Phone No: 04998275037 Email ...",Aided General,ManjeswaramManjeswara,Kasargode,Manjeshwar,139,Co-Education,15
102,14301,"N.A. MODEL HSS, CHENGALA Phone No: 04994256137...",Un-Aided,ChengalaKasaragod,Kasargode,Kasaragod,139,Co-Education,123
103,14302,"ESSA HIGHER SECONDARY SCHOOL, KUMBALA, KASARGO...",Un-Aided,KumbalaKasaragod,Kasargode,Manjeshwar,13439,Co-Education,12
104,14303,"MUHIMMATH HIGHER SECONDARY SCHOOL , MUHIMMATH ...",Un-Aided,PuthigeKasaragod,Kasargode,Manjeshwar,139,Co-Education,15


In [176]:
#splitting column with phone and email
master[['school_details', 'phone','email']] = master['School Name, Address, Phone No'].str.split('Phone No: | Email ID:', expand=True)

In [177]:
#extracting relevent infor
schools_kerala = master[['school_details', 'phone','email']]

In [63]:
#list of Public domains
public_domains = ('gmail.com', 'yahoo.com', 'yahoo.in', 'hotmail.com', 
                  'live.com','gmail .com', 'gmai.com', 'gmail.comcom', 
                  'rocketmail.com','ymail.com', 'google.com', 'rediffmail.com', 
                  'yahoo.co.in','gail.com', 'rediff.com', 'live.in' )

In [225]:
#emails to lower case
schools_kerala['email'] = schools_kerala['email'].str.lower()

C:\Users\pk\AppData\Local\Temp/ipykernel_22240/2412576529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_kerala['email'] = schools_kerala['email'].str.lower().str.strip()


In [231]:
#filtering schools based on public/private domains
kerala_pub_domain = schools_kerala[schools_kerala['email'].str.endswith(public_domains)]
kerala_pvt_domain = schools_kerala[~schools_kerala['email'].str.endswith(public_domains)]

#Saving to CSV
kerala_pub_domain.to_csv('kerala_pub_domain.csv')
kerala_pvt_domain.to_csv('kerala_pvt_domain.csv')

## CBSE School List

In [26]:
cbse_url = "https://saras.cbse.gov.in/SARAS/AffiliatedList/ListOfSchdirReport"

#Loading Page
driver.get(cbse_url)

#Clicking Radio Button
driver.find_element('xpath', '/html/body/div[2]/div[1]/form/div/div[2]/div/label[3]').click()

#Selecting State Name
Select(driver.find_element('name', 'State')).select_by_visible_text("KERALA")

#Clicking Search
driver.find_element('xpath', '/html/body/div[2]/div[1]/form/div/div[9]/div/input').click()

#Selecting 100 View
Select(driver.find_element('name', 'myTable_length')).select_by_visible_text("100")

soup = BeautifulSoup(driver.page_source, 'html.parser')

#Extracting Table
cbse_html = soup.find('table', {'id': 'myTable'})
cbse_pd = pd.read_html(str(cbse_html))[0]

In [34]:
cbse_pd

,S No,Aff. No,State & District,Status,School & Head Name,"Address,Phone & Email",Details
0,1,900001,State : KERALA District : ERNAKULAM,Senior Secondary Level,Name : KENDRIYA VIDYALAYA NO.1 Head/Principal ...,"Address : NAVAL BASE KATARIBAGH COCHIN, ERNAKU...",View
1,2,900002,State : KERALA District : ERNAKULAM,Senior Secondary Level,Name : KENDRIYA VIDYALAYA Head/Principal Name:...,Address : KADAVANTHRA (PO) ERNAKULAM COCHIN CO...,View
2,3,900003,State : KERALA District : ERNAKULAM,Senior Secondary Level,Name : KENDRIYA VIDYALAYA NO.2 Head/Principal ...,"Address : NAVAL BASE KOCHI, DISTT ERNAKULAM KE...",View
3,4,900004,State : KERALA District : THIRUVANANTHAPURAM,Senior Secondary Level,Name : KENDRIYA VIDYALAYA Head/Principal Name:...,Address : PATTOM TRIVANDRUM KERALA Phone No : ...,View
4,5,900005,State : KERALA District : KOZHIKODE,Senior Secondary Level,Name : KENDRIYA VIDYALAYA NO.1 Head/Principal ...,"Address : EAST HILL,(WEST HILL POST) CALICUT, ...",View
...,...,...,...,...,...,...,...
95,96,930064,State : KERALA District : ERNAKULAM,Senior Secondary Level,Name : ST. PETERS SENIOR SECONDARY SCHOOL Head...,Address : KADAYIRIPPU P.O KOLENCHERRY VIA ERNA...,View
96,97,930065,State : KERALA District : THIRUVANANTHAPURAM,Senior Secondary Level,Name : CHINMAYA VIDYALAYA Head/Principal Name:...,Address : MANACAUD - ATTUKAL THIRUVANANTHAPURA...,View
97,98,930066,State : KERALA District : ALAPPUZHA,Senior Secondary Level,Name : SREE NARAYANA CENTRAL SCHOOL Head/Princ...,"Address : KAYAMKULAM, DISTT ALAPPUZHA, KERALA ...",View
98,99,930067,State : KERALA District : THRISSUR,Senior Secondary Level,Name : AMRITA VIDYALAYAM Head/Principal Name: ...,Address : KODUNGALUR THRISSUR DIST. KERALA Pho...,View


In [35]:
cbse_master = cbse_pd
#iterating over next 14 pages
for i in range(14):
    #selecting next page
    driver.find_element('xpath', '//*[@id="myTable_next"]').click()
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #scraping html table to pd.dataframe
    cbse_html = soup.find('table', {'id': 'myTable'})
    cbse_pd = pd.read_html(str(cbse_html))[0]
    cbse_master = cbse_master.append(cbse_pd)
    

In [36]:
cbse_master

,S No,Aff. No,State & District,Status,School & Head Name,"Address,Phone & Email",Details
0,1,900001,State : KERALA District : ERNAKULAM,Senior Secondary Level,Name : KENDRIYA VIDYALAYA NO.1 Head/Principal ...,"Address : NAVAL BASE KATARIBAGH COCHIN, ERNAKU...",View
1,2,900002,State : KERALA District : ERNAKULAM,Senior Secondary Level,Name : KENDRIYA VIDYALAYA Head/Principal Name:...,Address : KADAVANTHRA (PO) ERNAKULAM COCHIN CO...,View
2,3,900003,State : KERALA District : ERNAKULAM,Senior Secondary Level,Name : KENDRIYA VIDYALAYA NO.2 Head/Principal ...,"Address : NAVAL BASE KOCHI, DISTT ERNAKULAM KE...",View
3,4,900004,State : KERALA District : THIRUVANANTHAPURAM,Senior Secondary Level,Name : KENDRIYA VIDYALAYA Head/Principal Name:...,Address : PATTOM TRIVANDRUM KERALA Phone No : ...,View
4,5,900005,State : KERALA District : KOZHIKODE,Senior Secondary Level,Name : KENDRIYA VIDYALAYA NO.1 Head/Principal ...,"Address : EAST HILL,(WEST HILL POST) CALICUT, ...",View
...,...,...,...,...,...,...,...
56,1457,980002,State : KERALA District : THIRUVANANTHAPURAM,Senior Secondary Level,Name : SAINIK SCHOOL Head/Principal Name: COMM...,Address : KAZHAKOOTAM SAINIK SCHOOL P.O DISTT....,View
57,1458,980004,State : KERALA District : ERNAKULAM,Senior Secondary Level,Name : NAVY CHILDREN SCHOOL Head/Principal Nam...,"Address : NAVAL BASE COCHIN, DISTT ERNAKULAM, ...",View
58,1459,980005,State : KERALA District : KANNUR,Senior Secondary Level,Name : ARMY PUBLIC SCHOOL Head/Principal Name:...,Address : BURNACHERRY POST CANNANORE KERALA Ph...,View
59,1460,980007,State : KERALA District : THIRUVANANTHAPURAM,Senior Secondary Level,Name : ARMY PUBLIC SCHOOL Head/Principal Name:...,"Address : pangode, thirumala po, trivandrum, k...",View


In [49]:
#Splitting columns
cbse_master[["0", "State","District"]] = cbse_master['State & District'].str.split('State : | District : ', expand=True)

cbse_master[["0", "School", "Head"]] = cbse_master['School & Head Name'].str.split('Name : | Head/Principal Name: ', expand=True)

cbse_master[["0", "Address", "Phone", "Email", "Website"]] = cbse_master['Address,Phone & Email'].str.split('Address : |Phone No : |Email :|Website :', expand=True)

In [57]:
cbse_master.columns

Index(['S No', 'Aff. No', 'State & District', 'Status', 'School & Head Name',
       'Address,Phone & Email', 'Details', '0', 'State', 'District', 'School',
       'Head', 'Address', 'Phone', 'Email', 'Website'],
      dtype='object')

In [61]:
#Selecting Relevent Info
cbse_kerala = cbse_master[['Aff. No', 'School', 'Address', 'Email', 'Phone', 'Website','State', 'District']]

In [62]:
cbse_kerala.head()

,Aff. No,School,Address,Email,Phone,Website,State,District
0,900001,KENDRIYA VIDYALAYA NO.1,"NAVAL BASE KATARIBAGH COCHIN, ERNAKULAM DISTT....",kv1navalbasekochi@gmail.com,"0484, 2668472,",http://www.kv1kochi.org,KERALA,ERNAKULAM
1,900002,KENDRIYA VIDYALAYA,KADAVANTHRA (PO) ERNAKULAM COCHIN COCHIN KERALA,sudhasurabio@gmail.com,"0484, 2204785,",www.cstcvp.g,KERALA,ERNAKULAM
2,900003,KENDRIYA VIDYALAYA NO.2,"NAVAL BASE KOCHI, DISTT ERNAKULAM KERALA",minu8_2003@yahoo.com,"0484, 2668344, 668337",,KERALA,ERNAKULAM
3,900004,KENDRIYA VIDYALAYA,PATTOM TRIVANDRUM KERALA,kendriyavidyalaya_pattom@yahoo.co.in,"0471, 2445854,",,KERALA,THIRUVANANTHAPURAM
4,900005,KENDRIYA VIDYALAYA NO.1,"EAST HILL,(WEST HILL POST) CALICUT, DISTT KOZH...",kvcalicutone@gmail.com,"0495, 2382299,",http://www.kvcalicutone.o,KERALA,KOZHIKODE


In [68]:
#Removing White spaces in email address
cbse_kerala['Email'] = cbse_kerala['Email'].str.strip()

#filtering public domain and private domain emails
cbse_kerala_pub_domain = cbse_kerala[cbse_kerala['Email'].str.endswith(public_domains)]
cbse_kerala_pvt_domain = cbse_kerala[~cbse_kerala['Email'].str.endswith(public_domains)]

cbse_kerala_pvt_domain.to_csv('cbse_kerala_pvt_domain.csv')
cbse_kerala_pub_domain.to_csv('cbse_kerala_pub_domain.csv')

C:\Users\pk\AppData\Local\Temp/ipykernel_7092/4152858891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbse_kerala['Email'] = cbse_kerala['Email'].str.strip()
